In [1]:
import os
import cv2
import time
import mediapipe as mp
import uuid
import keyboard
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
label=["batu","kertas","guting"]
numofimage=20
hashmap={"batu":0,"kertas":1,"guting":2}

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [4]:
for labe in label:
    os.mkdir("./images/{}".format(labe))

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './images/batu'

In [89]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    for labe in label:
        print("mencari gambar {}".format(labe))
        z=0
        while z<numofimage:
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue
            image.flags.writeable= False
            mpimg=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            results=hands.process(mpimg)
            image.flags.writeable = True
            mpimg = cv2.cvtColor(mpimg, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(
                            mpimg,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS,
                            mp_drawing_styles.get_default_hand_landmarks_style(),
                            mp_drawing_styles.get_default_hand_connections_style())
            cv2.imshow('keluar',mpimg)
            if keyboard.is_pressed('y'):
                z=z+1
                print("images/{}/{}.jpg".format(labe,"gambar"+str(z)+labe))
                cv2.imwrite("images/{}/{}.jpg".format(labe,"gambar"+str(z)+labe),image)
                time.sleep(1)
            if cv2.waitKey(5) & 0xFF == ord('q'):
                cap.release()
                cv2. destroyAllWindows()
                break
cap.release()
cv2. destroyAllWindows()

mencari gambar guting
images/guting/gambar1guting.jpg


KeyboardInterrupt: 

In [63]:
cv2. destroyAllWindows()

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [5]:
array=np.array([],ndmin=3)
df_y=pd.DataFrame({'y':pd.Series(dtype='int')})


In [6]:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    for k in range(numofimage):
        for idx, file in enumerate(label):
            # Read an image, flip it around y-axis for correct handedness output (see
            # above).
            image = cv2.flip(cv2.imread("images/{}/{}.jpg".format(file,"gambar"+str(9)+file)),1)
            # Convert the BGR image to RGB before processing.
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            # Print handedness and draw hand landmarks on the image.
            if not results.multi_hand_landmarks:
                continue
            image_height, image_width, _ = image.shape
            annotated_image = image.copy()
            temparray=np.array([[i.x,i.y,i.z] for i in results.multi_hand_world_landmarks[0].landmark])
            temparray=temparray.reshape((1,63))
            array=np.append(array,[temparray])
            df_y.loc[len(df_y)]={'y':hashmap[file]}
    # Draw hand world landmarks.
        


In [7]:
df_x=pd.DataFrame(array.reshape(array.shape[0]//63,63))
df_xinput=array.reshape(array.shape[0]//63,63)
print(df_xinput)

[[ 0.06999166  0.04428028  0.04478407 ... -0.01150896  0.03958318
   0.04458234]
 [ 0.01418806  0.08254506  0.01956361 ...  0.03543501 -0.04641523
  -0.04385572]
 [ 0.0517999   0.06940232 -0.00427478 ...  0.02612365  0.02263223
  -0.03442078]
 ...
 [ 0.06999166  0.04428028  0.04478407 ... -0.01150896  0.03958318
   0.04458234]
 [ 0.01418806  0.08254506  0.01956361 ...  0.03543501 -0.04641523
  -0.04385572]
 [ 0.0517999   0.06940232 -0.00427478 ...  0.02612365  0.02263223
  -0.03442078]]


In [8]:

df_yinput=np.array(df_y)
df_yinput=df_yinput.flatten()

In [9]:
X_train,X_test,y_train,y_test=train_test_split(
    df_xinput,df_yinput,test_size=0.33,random_state=42
)

In [10]:
clf = RandomForestClassifier(max_depth=10, random_state=6,n_estimators=20)

In [11]:
clf.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, n_estimators=20, random_state=6)

In [12]:
print(clf.score(X_test,y_test))

1.0


In [13]:
print(clf.score(X_train,y_train))

1.0


In [21]:
cap=cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
org = (50, 50)
color = (0, 0, 0)
thickness = 2
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5) as hands:
    while True:
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue
        image.flags.writeable= False
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        results=hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style())
            # temparray=np.array([[i.x,i.y,i.z] for i in results.multi_hand_world_landmarks[0].landmark])
            # temparray=temparray.reshape((1,63))
            # print(temparray)
            # print(clf.predict(temparray))
            # image = cv2.putText(image,label[clf.predict(temparray)],org,font,fontScale,color,cv2.LINE_AA)

        cv2.imshow('keluar',image)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            cap.release()
            cv2. destroyAllWindows()
            break
cap.release()
cv2. destroyAllWindows()
        

KeyboardInterrupt: 

In [22]:
cap.release()
cv2. destroyAllWindows()
        